# Resale Prices Working File

In [1]:
import requests
import os
import pandas as pd
import numpy as np
from tqdm import trange

In [4]:
pd.set_option('display.width', 300)
pd.set_option('display.max_columns', 10)

os.chdir('D:/User/Documents/R/Github/Portfolio/hdb_resale_prices/')

In [151]:
def onemap_geocoding(address):
    result = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    result = eval(result.text)
    
    if result['found'] > 0:
        lat, long = result['results'][0]['LATITUDE'], result['results'][0]['LONGITUDE']
        return address, lat, long
    else:
        pass

def onemap_geocoding_flat(address):
    result = requests.get('https://developers.onemap.sg/commonapi/search?searchVal='+address+'&returnGeom=Y&getAddrDetails=Y&pageNum=1')
    result = eval(result.text)
    
    if result['found'] > 0:
        for i in result['results']:
            lat, long = i['LATITUDE'], i['LONGITUDE']
            r_block, r_street = i['BLK_NO'], i['ROAD_NAME']
            r_add = r_block + ' ' + r_street
            
            if r_add == address:
                return address, r_add, lat, long
            else:
                continue
        return address, 'none', '0', '0'
    else:
        pass
    
def geocode_series(addresses, func):
    coord_list = []
    failed_list = []
    failed = 0
    t = trange(len(addresses), desc='Geoding Addresses')
    for i in t:
        try:
            result = func(addresses[i])
            if len(result) > 0:
                coord_list.append(result)
                t.set_description('{} failed'.format(failed))
        except:
            failed_list.append(addresses[i])
            failed += 1
            t.set_description('{} failed'.format(failed))
    print(failed_list if failed_list else "All passed")
    return coord_list

def calculate_dist(address_df, dist_df, check_date=False):
    temp = address_df.copy()
    for _, i in dist_df.iterrows():
        name, lat, long = i[0], float(i[1]), float(i[2])
        distances = (((address_df['latitude'] - lat) * 110.574) ** 2 + ((address_df['longitude'] - long) * 111.32) ** 2) ** 0.5
        if check_date:
            exists = address_df['sale_year'] >= dist_df[check_date] & address_df['sale_month'] >= dist_df[check_date]
        temp = pd.concat([temp, distances.rename(name), exists.rename(name + '_exists')], axis=1)
    return temp
    nearest_name = temp.drop(['address', 'longitude', 'latitude'], axis=1).idxmin(axis=1)
    nearest_dist = temp.drop(['address', 'longitude', 'latitude'], axis=1).min(axis=1)
    nearest = pd.concat([temp['address'], nearest_name, nearest_dist], axis=1)
    return nearest

# test = calculate_dist(resale_df, mrt_coord_df)

def calculate_pri_score(address_df, dist_list, rank_list, dist_cutoff, num_cutoff, dist_weights={1: 0.1, 2: 0.5, 999: 1}):
    temp = address_df.copy()
    for i in dist_list:
        name, lat, long = i[0], float(i[1]), float(i[2])
        rank = int(rank_list.loc[rank_list['pri_sch_name'] == name]['rank'])
        temp[str(rank)] = (((address_df['latitude'] - lat) * 110.574) ** 2 + ((address_df['longitude'] - long) * 111.32) ** 2) ** 0.5
        
    t = trange(len(temp), desc='Address')
    distances = temp.drop(['address', 'latitude', 'longitude'], axis=1)
    result = {}
    for i in t:
        row = distances.iloc[i]
        row = row[row <= dist_cutoff][:num_cutoff]
        
        score = 0
        for j in row.index:
            dist = row[j]
            for ring in dist_weights.keys():
                if dist < ring:
                    weight = dist_weights[ring]
                    break
            score += (weight * int(j))

        result[temp.iloc[i]['address']] = score if score != 0 else 9999
        
    result = pd.DataFrame.from_dict(result, orient='index', columns=['pri_sch_score'])
    return result

## Dataset

Importing resale prices from data.gov API

In [ ]:
dataset = requests.get('https://data.gov.sg/api/action/datastore_search?resource_id=f1765b54-a209-4718-8d38-a39237f502b3&limit=9999999').json()
resale_df = pd.DataFrame(dataset['result']['records'])
resale_df.to_csv('data/resale_df_raw.csv', index=False)

In [92]:
resale_df = pd.read_csv('data/resale_df_raw.csv')

In [93]:
resale_df.head()

,town,flat_type,flat_model,floor_area_sqm,street_name,...,remaining_lease,lease_commence_date,storey_range,_id,block
0,ANG MO KIO,2 ROOM,Improved,44.0,ANG MO KIO AVE 10,...,61 years 04 months,1979,10 TO 12,1,406
1,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 4,...,60 years 07 months,1978,01 TO 03,2,108
2,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,...,62 years 05 months,1980,01 TO 03,3,602
3,ANG MO KIO,3 ROOM,New Generation,68.0,ANG MO KIO AVE 10,...,62 years 01 month,1980,04 TO 06,4,465
4,ANG MO KIO,3 ROOM,New Generation,67.0,ANG MO KIO AVE 5,...,62 years 05 months,1980,01 TO 03,5,601


Getting search fields for geocoding

In [99]:
resale_df['address'] = resale_df['block'] + ' ' + resale_df['street_name']
# resale_df['address'] = resale_df['address'].str.replace('ST. GEORGE', 'SAINT GEORGE')

changes = [[r'\bAVE\b', 'AVENUE'],
           [r'\bRD\b', 'ROAD'],
           [r'\bNTH\b', 'NORTH'],
           [r'\bSTH\b', 'SOUTH'],
           [r'\bST\. GEORGE\b', 'SAINT GEORGE'],
           [r'\bST\b', 'STREET'],
           [r'\bCTRL\b', 'CENTRAL'],
           [r'\bDR\b', 'DRIVE'],
           [r'\bBT\b', 'BUKIT'],
           [r'\bJLN\b', 'JALAN'],
           [r'\bCRES\b', 'CRESCENT'],
           [r'\bLOR\b', 'LORONG'],
           [r"\bC'WEALTH\b", 'COMMONWEALTH'],
           [r'\bCL\b', 'CLOSE'],
           [r'\bGDNS\b', 'GARDENS'],
           [r'\bUPP\b', 'UPPER'],
           [r'\bHTS\b', 'HEIGHTS'],
           [r'\bTER\b', 'TERRACE'],
           [r'\bPL\b', 'PLACE'],
           [r'\bTG\b', 'TANJONG'],
           [r'\bPK\b', 'PARK'],
           [r'\bMKT\b', 'MARKET'],
           [r'\bKG\b', 'KAMPONG']]
for short, long in changes:
    resale_df['address'] = resale_df['address'].str.replace(short, long, regex=True)

Geocoding address with OneMap API

In [44]:
house_addresses = resale_df['address'].unique()
coord_list = geocode_series(house_addresses, onemap_geocoding_flat)
coord_df = pd.DataFrame(coord_list, columns=['address', 'search_address', 'latitude', 'longitude'])

0 failed: 100%|██████████| 9248/9248 [15:30<00:00,  9.94it/s]

All passed


In [83]:
none_check = coord_df.isnull().sum()
none_check

address      0
latitude     0
longitude    0
dtype: int64

In [47]:
cck_exception = eval(requests.get('https://developers.onemap.sg/commonapi/search?searchVal=BLK 216 AND 215 CHOA CHU KANG CENTRAL&returnGeom=Y&getAddrDetails=Y&pageNum=1').text)['results'][0]
for idx in coord_df[coord_df['search_address'] == 'none'].index:
    coord_df.loc[idx]['latitude'] = cck_exception['LATITUDE']
    coord_df.loc[idx]['longitude'] = cck_exception['LONGITUDE']

In [95]:
coord_df[['latitude','longitude']] = coord_df[['latitude', 'longitude']].astype(float)
coord_df.drop('search_address', axis=1, inplace=True)

KeyError: "['search_address'] not found in axis"

In [100]:
resale_df = resale_df.merge(right=coord_df, on='address', how='left')
resale_map_df = resale_df[['address', 'month', 'latitude', 'longitude']]
resale_map_df['sale_year'] = resale_map_df['month'].str[:4].astype(int)
resale_map_df['sale_month'] = resale_map_df['month'].str[5:7].astype(int)

In [117]:
resale_map_df.head()

,address,month,latitude,longitude,sale_year,sale_month
0,406 ANG MO KIO AVENUE 10,2017-01,1.362005,103.853880,2017,1
1,108 ANG MO KIO AVENUE 4,2017-01,1.370943,103.837975,2017,1
2,602 ANG MO KIO AVENUE 5,2017-01,1.380709,103.835368,2017,1
3,465 ANG MO KIO AVENUE 10,2017-01,1.366201,103.857201,2017,1
4,601 ANG MO KIO AVENUE 5,2017-01,1.381041,103.835132,2017,1


In [ ]:
resale_df.to_csv('data/resale_df_geocode.csv', index=False)

## MRT Stations

In [120]:
mrt_df = pd.read_csv('data\mrt_stations.csv').query('is_active == "Y"')
mrt_df['commence_date'] = pd.to_datetime(mrt_df['commence_date'], format='%Y-%m-%d')

In [124]:
mrt_df.head()

,stn_id,line_id,stn_name,commence_date,is_active
0,BP1,BPLRT,CHOA CHU KANG LRT STATION,1999-11-06,Y
1,BP2,BPLRT,SOUTH VIEW LRT STATION,1999-11-06,Y
2,BP3,BPLRT,KEAT HONG LRT STATION,1999-11-06,Y
3,BP4,BPLRT,TECK WHYE LRT STATION,1999-11-06,Y
4,BP5,BPLRT,PHOENIX LRT STATION,1999-11-06,Y


Geocoding MRT stations

In [161]:
# mrt_names = mrt_df['stn_name'].unique()
# mrt_coord = geocode_series(mrt_names, onemap_geocoding)
mrt_coord_df = pd.DataFrame(mrt_coord, columns = ['stn_name', 'latitude', 'longitude'])
mrt_coord_df[['latitude', 'longitude']] = mrt_coord_df[['latitude', 'longitude']].astype(float)
mrt_coord_df = mrt_df.merge(right=mrt_coord_df, on='stn_name', how='left')

In [163]:
mrt_coord_df

,stn_id,line_id,stn_name,commence_date,is_active,latitude,longitude
0,BP1,BPLRT,CHOA CHU KANG LRT STATION,1999-11-06,Y,1.384755,103.744538
1,BP2,BPLRT,SOUTH VIEW LRT STATION,1999-11-06,Y,1.380298,103.745292
2,BP3,BPLRT,KEAT HONG LRT STATION,1999-11-06,Y,1.378603,103.749056
3,BP4,BPLRT,TECK WHYE LRT STATION,1999-11-06,Y,1.376685,103.753712
4,BP5,BPLRT,PHOENIX LRT STATION,1999-11-06,Y,1.378615,103.757996
...,...,...,...,...,...,...,...
198,TE17,TEL,OUTRAM PARK MRT STATION,2022-11-13,Y,1.279740,103.839514
199,TE18,TEL,MAXWELL MRT STATION,2022-11-13,Y,1.280533,103.843884
200,TE19,TEL,SHENTON WAY MRT STATION,2022-11-13,Y,1.277717,103.850368
201,TE20,TEL,MARINA BAY MRT STATION,2022-11-13,Y,1.276427,103.854598


In [84]:
none_check = mrt_coord_df.isnull().sum()
none_check

mrt_name     0
latitude     0
longitude    0
dtype: int64

In [153]:
mrt_dist_df = calculate_dist(resale_map_df, mrt_coord_df, check_date='commence_date')

KeyError: 'commence_date'

In [154]:
mrt_dist_df

,address,latitude,longitude,CHOA CHU KANG LRT STATION,SOUTH VIEW LRT STATION,...,GREAT WORLD MRT STATION,HAVELOCK MRT STATION,MAXWELL MRT STATION,SHENTON WAY MRT STATION,GARDENS BY THE BAY MRT STATION
0,406 ANG MO KIO AVENUE 10,1.362005,103.853880,12.429220,12.256109,...,7.913874,8.435262,9.077069,9.328227,9.357661
1,108 ANG MO KIO AVENUE 4,1.370943,103.837975,10.512957,10.369205,...,8.595387,9.128529,10.018550,10.400265,10.736982
2,602 ANG MO KIO AVENUE 5,1.380709,103.835368,10.121150,10.027411,...,9.664694,10.197250,11.117297,11.509994,11.853991
3,465 ANG MO KIO AVENUE 10,1.366201,103.857201,12.708375,12.554872,...,8.466052,8.984341,9.587929,9.813580,9.765800
4,601 ANG MO KIO AVENUE 5,1.381041,103.835132,10.093287,10.001320,...,9.701052,10.233535,11.156200,11.550211,11.896991
...,...,...,...,...,...,...,...,...,...,...,...
9243,502C YISHUN STREET 51,1.417323,103.841554,11.384443,11.471349,...,13.738313,14.271499,15.127618,15.467970,15.620043
9244,513A YISHUN STREET 51,1.416040,103.843489,11.545714,11.623847,...,13.611912,14.145072,14.983597,15.314114,15.441968
9245,513D YISHUN STREET 51,1.415634,103.844328,11.621505,11.696568,...,13.574752,14.107857,14.938658,15.264819,15.381793
9246,673A YISHUN AVENUE 4,1.419999,103.841999,11.528074,11.626098,...,14.036751,14.569942,15.422725,15.760283,15.902055


In [145]:
resale_map_df

,address,month,latitude,longitude,sale_year,sale_month
0,406 ANG MO KIO AVENUE 10,2017-01,1.362005,103.853880,2017,1
1,108 ANG MO KIO AVENUE 4,2017-01,1.370943,103.837975,2017,1
2,602 ANG MO KIO AVENUE 5,2017-01,1.380709,103.835368,2017,1
3,465 ANG MO KIO AVENUE 10,2017-01,1.366201,103.857201,2017,1
4,601 ANG MO KIO AVENUE 5,2017-01,1.381041,103.835132,2017,1
...,...,...,...,...,...,...
126381,836 YISHUN STREET 81,2022-04,1.415452,103.833091,2022,4
126382,828 YISHUN STREET 81,2022-05,1.415191,103.832902,2022,5
126383,633 YISHUN STREET 61,2022-01,1.418658,103.839842,2022,1
126384,633 YISHUN STREET 61,2022-02,1.418658,103.839842,2022,2
